<a href="https://colab.research.google.com/github/yaoshiang/The-Real-World-Weight-Crossentropy-Loss-Function/blob/master/Churn_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Churn predictor

## This tool uses 5 months of churn to predict the churn and subscriber totals for the full 120 months. 

(c) 2020 Yaoshiang Ho



```
# This is formatted as code
```

Enter your six months of end-of-month subscriber numbers here, including month 0 (all sub adds prior to end of month). 

As a sample, we used 1000, 600, 420, 315, 252, and 215. This represents churn of 40%, 30%, 25%, 20%, and 15%. 



In [0]:
subs=[1.000, .600, .420, .315, .252, .215]

This is the churn rate per month:

In [45]:
import numpy as np

subs = np.array(subs)

churn = (subs[0:5] - subs[1:6]) / subs[0:5]
print(churn)

[0.4       0.3       0.25      0.2       0.1468254]


Setup 99 segments with churn ranging from 1% to 99%


In [46]:
segment_churns = np.arange(.01,1., 0.01)
print(segment_churns)

[0.01 0.02 0.03 0.04 0.05 0.06 0.07 0.08 0.09 0.1  0.11 0.12 0.13 0.14
 0.15 0.16 0.17 0.18 0.19 0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28
 0.29 0.3  0.31 0.32 0.33 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41 0.42
 0.43 0.44 0.45 0.46 0.47 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55 0.56
 0.57 0.58 0.59 0.6  0.61 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7
 0.71 0.72 0.73 0.74 0.75 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84
 0.85 0.86 0.87 0.88 0.89 0.9  0.91 0.92 0.93 0.94 0.95 0.96 0.97 0.98
 0.99]


In [0]:
import keras.backend as K

months = 5

def fprop(segment_sizes):
  '''
  Arguments: 
  * segment_sizes: numpy array of size (99)
  * segment_churns: numpy array of size (99)
  Returns:
  * np array shape (months, total subs)
  '''
  retval = []
  # retval2 = []
  current_month_sizes = segment_sizes
  for i in range(5):
    current_month_sizes = current_month_sizes * (1. - segment_churns) # m,99 * 99 = m,99
    retval.append(K.sum(current_month_sizes, axis=-1)) # m
    # retval2.append(current_month_sizes)

  retval = K.stack(retval, axis=-1) # m,99,6

  return retval

In [0]:
from keras.models import Model
from keras.layers import Lambda, Input, Dense
from keras.utils import to_categorical
from keras import optimizers
from keras import regularizers

kx = Input(shape=(1,))
kd = Dense(99, use_bias=False, activation='softmax', activity_regularizer=regularizers.l2())(kx) 
k_y = Lambda(fprop, output_shape=(5,))(kd)
m = Model(inputs=kx, outputs=k_y)

In [64]:
opt=optimizers.Adam(lr=.1)

m.compile(optimizer=opt, loss='mse')

x = np.ones(shape=(1000,1))
y = np.tile(subs[1:], (1000,1))
m.fit(x=x, y=y, epochs=5, batch_size=1)

Epoch 1/5
1000/1000 [==============================] - 2s 2ms/step - loss: 1.3962e-04
Epoch 2/5
1000/1000 [==============================] - 1s 1ms/step - loss: 1.2201e-04
Epoch 3/5
1000/1000 [==============================] - 1s 1ms/step - loss: 1.2199e-04
Epoch 4/5
1000/1000 [==============================] - 1s 1ms/step - loss: 1.2212e-04
Epoch 5/5
1000/1000 [==============================] - 1s 1ms/step - loss: 1.2231e-04


In [65]:
r = m.predict(x=np.ones(shape=(1)))
print(r)

[[0.5970495  0.41555873 0.31506526 0.2522746  0.20971647]]


In [66]:
l = m.layers[1].get_weights()
# print(l[0])

sizes = K.eval(K.softmax(l[0]))
print(sizes)

[[0.01385197 0.01361397 0.01340884 0.01323502 0.0130723  0.012945
  0.01283998 0.01275359 0.01267333 0.01263016 0.01258475 0.01256928
  0.01254031 0.01254168 0.01256009 0.01257701 0.01259918 0.01262729
  0.01265678 0.01270981 0.01274777 0.01279294 0.01284428 0.01290018
  0.0129462  0.01300683 0.01305353 0.01310069 0.01316388 0.01323209
  0.01330729 0.01339382 0.01344502 0.0134765  0.01350727 0.01350668
  0.01348353 0.0134785  0.0134561  0.01343746 0.01339937 0.01335076
  0.0132979  0.01323189 0.01316326 0.01307827 0.01298946 0.01288926
  0.01277972 0.01266405 0.01253709 0.01240268 0.01225634 0.01210816
  0.01194583 0.01177765 0.01159874 0.01141369 0.0112179  0.01101848
  0.01081184 0.01059447 0.01037573 0.01014719 0.00990963 0.00966922
  0.00942565 0.00916939 0.00891543 0.00865699 0.00839307 0.00812005
  0.00785226 0.00757655 0.007299   0.00701353 0.0067318  0.00644793
  0.00617117 0.00588707 0.00560199 0.00531335 0.00503239 0.00475024
  0.00447251 0.00418999 0.00391332 0.00364386 0.00

In [0]:
def build(sizes, churn_segs, months=120):
  '''
  returns
  * size of cohort for each month
  * detail per churn segment
  * churns
  * estimated lifetime of each sub in months. 
  '''

  initial_subs = np.sum(sizes)

  summary = np.zeros(shape=(months+1))
  detail = np.zeros(shape=(months+1, churn_segs.shape[0]))
  churns = np.zeros(shape=(months+1))

  current = sizes
  detail[0] = current

  # Forward prop through the months 
  for i in range(1, months+1):
    current = current * (1. - churn_segs)
    detail[i] = current

  summary = np.sum(detail, axis=1)

  # Calculate churns 
  churns[0] = 0
  churns[1:] = (summary[:-1] - summary[1:]) / summary[:-1]

  lt = np.sum(summary) / initial_subs

  return summary, detail, churns, lt

summary, detail, churns, lt = build(sizes, segment_churns)

This is the chance that a sub is still around in any given month. 

In [68]:
print(summary)

[1.00000012 0.59704949 0.41555873 0.31506525 0.25227459 0.20971646
 0.17912652 0.15614285 0.13826802 0.12397866 0.11229713 0.10256924
 0.0943417  0.08729096 0.08118013 0.07583191 0.07111101 0.06691242
 0.06315339 0.05976787 0.05670243 0.05391345 0.0513649  0.0490268
 0.04687399 0.04488517 0.04304222 0.04132959 0.03973391 0.03824355
 0.03684842 0.03553965 0.03430946 0.033151   0.03205817 0.03102556
 0.03004834 0.02912219 0.02824322 0.02740793 0.02661318 0.02585609
 0.02513407 0.02444476 0.02378601 0.02315585 0.02255247 0.02197424
 0.02141962 0.02088722 0.02037575 0.01988401 0.01941092 0.01895543
 0.01851661 0.01809358 0.01768551 0.01729165 0.01691127 0.01654371
 0.01618836 0.01584463 0.01551197 0.01518986 0.01487784 0.01457545
 0.01428225 0.01399786 0.01372189 0.01345399 0.01319381 0.01294105
 0.0126954  0.01245657 0.0122243  0.01199834 0.01177842 0.01156434
 0.01135587 0.01115279 0.01095492 0.01076207 0.01057405 0.0103907
 0.01021184 0.01003734 0.00986703 0.00970077 0.00953844 0.009379

In [69]:
print (detail)

[[1.38519658e-002 1.36139710e-002 1.34088360e-002 ... 1.41583383e-003
  1.22111384e-003 9.99703305e-004]
 [1.37134461e-002 1.33416915e-002 1.30065709e-002 ... 4.24750149e-005
  2.44222768e-005 9.99703305e-006]
 [1.35763117e-002 1.30748577e-002 1.26163738e-002 ... 1.27425045e-006
  4.88445535e-007 9.99703305e-008]
 ...
 [4.23120797e-003 1.25505209e-003 3.68515468e-004 ... 2.82696442e-183
  4.05784675e-204 9.99703305e-240]
 [4.18889589e-003 1.22995105e-003 3.57460004e-004 ... 8.48089326e-185
  8.11569350e-206 9.99703305e-242]
 [4.14700693e-003 1.20535203e-003 3.46736204e-004 ... 2.54426798e-186
  1.62313870e-207 9.99703305e-244]]


These are the churn rates per month. 

In [70]:
print(churns)

[0.         0.40295059 0.30397943 0.24182737 0.19929414 0.16869766
 0.14586331 0.1283097  0.11447744 0.10334534 0.0942221  0.08662633
 0.08021452 0.07473624 0.07000529 0.06588088 0.06225482 0.05904279
 0.05617823 0.05360796 0.05128902 0.04918636 0.04727112 0.04551934
 0.04391096 0.04242906 0.04105928 0.03978937 0.03860877 0.03750839
 0.03648033 0.03551767 0.03461438 0.03376514 0.03296523 0.0322105
 0.03149722 0.0308221  0.03018214 0.0295747  0.02899736 0.02844796
 0.02792452 0.02742526 0.02694856 0.02649293 0.02605703 0.0256396
 0.02523952 0.02485573 0.02448728 0.02413327 0.02379288 0.02346536
 0.02315    0.02284614 0.02255319 0.02227057 0.02199776 0.02173426
 0.02147962 0.02123342 0.02099523 0.0207647  0.02054147 0.02032521
 0.02011559 0.01991234 0.01971517 0.01952383 0.01933806 0.01915764
 0.01898234 0.01881197 0.01864631 0.01848519 0.01832843 0.01817585
 0.01802732 0.01788266 0.01774174 0.01760442 0.01747057 0.01734008
 0.01721281 0.01708865 0.01696751 0.01684927 0.01673385 0.016621

This is your expected months of lifetime per sub.

In [71]:
print(lt)

6.0026820880184655


This is your average churn for the segment. 

In [72]:
print(1/lt)

0.16659219751051454
